In [1]:
'''
Investigate 2016 presidential election survey
'''

import pandas as pd
import numpy as np
import gensim.models as g
import sys 

#parameters
doc2vec_dir = "../model/enwiki_dbow/doc2vec.bin"

In [2]:
df=pd.read_csv('data/anes_timeseries_2016_rawdata.txt',delimiter='|',low_memory=False)

In [27]:
df

,version,V160001,V160101,V160101f,V160101w,V160102,V160102f,V160102w,V160201,V160201f,...,V168123,V168124,V168125,V168126,V168127,V168128,V168129,V168130,V168131,V168132
0,ANES2016TimeSeries_20170502,300001,0.8270,0.8877,0.0000,0.8420,0.9271,0.0000,121,21,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,1
1,ANES2016TimeSeries_20170502,300002,1.0806,1.1605,0.0000,1.0133,1.0841,0.0000,123,23,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,1
2,ANES2016TimeSeries_20170502,300003,0.3878,0.4161,0.0000,0.3672,0.3985,0.0000,121,21,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,1
3,ANES2016TimeSeries_20170502,300004,0.3596,0.3852,0.0000,0.3663,0.4183,0.0000,118,18,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,1
4,ANES2016TimeSeries_20170502,300006,0.6470,0.6931,0.0000,0.6463,0.7262,0.0000,113,13,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,1
5,ANES2016TimeSeries_20170502,300007,0.7062,0.7588,0.0000,0.6881,0.7246,0.0000,104,4,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,1
6,ANES2016TimeSeries_20170502,300008,3.9604,4.2512,0.0000,4.6151,4.7902,0.0000,105,5,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,1
7,ANES2016TimeSeries_20170502,300012,0.9620,1.0325,0.0000,0.9426,1.0410,0.0000,104,4,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,1
8,ANES2016TimeSeries_20170502,300018,0.9761,1.0481,0.0000,1.0097,1.0733,0.0000,124,24,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,1
9,ANES2016TimeSeries_20170502,300020,0.6182,0.6642,0.0000,0.6005,0.6375,0.0000,121,21,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,1


In [3]:
# df[['V162007','V162008','V162065x']]

In [4]:
xls = pd.ExcelFile('data/anes_timeseries_2016_redacted_openends.xlsx')

In [6]:
open_response_dfs = []
sheet_names = xls.sheet_names
for name in sheet_names:
    open_response_df = xls.parse(name)
    open_response_dfs.append(open_response_df)

In [7]:
or_df=pd.concat(open_response_dfs,axis=1,ignore_index=True)

In [8]:
or_df=or_df.iloc[:,[0]+[i for i in range(1,58,2)]]

In [9]:
or_df.columns = ['id']+sheet_names

In [28]:
or_df.shape

(4271, 30)

In [19]:
str_df = or_df.iloc[:,1:].astype(str)

In [20]:
def represents_int(s):
    try: 
        int(s)
        return True
    except ValueError:
        return False

def replace_vals(s):
    if s == 'nan' or represents_int(s):
        return ''
    else:
        return s
    

In [32]:
str_df = str_df.applymap(replace_vals)
str_df = str_df.apply(lambda x: ' '.join(x), axis=1)

In [41]:
master = pd.concat([df,or_df,str_df],
                   axis=1,
                   ignore_index=True)
master.columns = list(df.columns)+list(or_df.columns)+['concat_text']

In [49]:
pretrained_model = g.Doc2Vec.load(doc2vec_dir)

In [50]:
master['weights'] = master['concat_text'].apply(pretrained_model.infer_vector)

In [54]:
weights = master['weights'].apply(pd.Series)

# rename each variable is tags
weights = weights.rename(columns = lambda x : 'weight_' + str(x))

# view the tags dataframe
master = pd.concat([master[:], weights[:]], axis=1)

In [56]:
master.to_csv('data/full_survey.csv')

In [58]:
selected = master[['V160001','V162007','V162008','V162065x','concat_text']+['weight_'+str(i) for i in range(300)]]

In [60]:
selected.to_csv('data/selected.csv')